# Google Colab으로 진행했습니다~

In [25]:
# Google Colab에서 GPU 확인
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [26]:
from model import InceptionV2  # 업로드된 파일에 따라 import 수정
model = InceptionV2(num_classes=10).to(device)
# 체크포인트 로드
start_epoch, _ = load_checkpoint(model, optimizer)

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV2의 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

# DataLoader 설정
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [27]:
import torch.optim as optim
import torch.nn as nn

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [28]:
# 훈련용 변수
num_epochs = 10
train_losses = []
train_accuracies = []

In [ ]:
# 훈련 루프
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    print(f"Epoch [{epoch+1}/{num_epochs}]")

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 기록
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # 배치마다 출력
        if (i + 1) % 10 == 0:  # 매 10배치마다 출력
            batch_loss = running_loss / (i + 1)
            batch_accuracy = 100 * correct / total
            print(f"Batch [{i+1}/{len(train_loader)}], Loss: {batch_loss:.4f}, Accuracy: {batch_accuracy:.2f}%")

    # Epoch 결과 저장
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

Epoch [1/10]
Batch [10/1563], Loss: 2.3154, Accuracy: 9.38%
Batch [20/1563], Loss: 2.3111, Accuracy: 10.62%
Batch [30/1563], Loss: 2.3064, Accuracy: 10.21%
Batch [40/1563], Loss: 2.3064, Accuracy: 10.16%
Batch [50/1563], Loss: 2.3065, Accuracy: 9.38%
Batch [60/1563], Loss: 2.3066, Accuracy: 9.64%
Batch [70/1563], Loss: 2.3063, Accuracy: 9.69%
Batch [80/1563], Loss: 2.3061, Accuracy: 9.38%
Batch [90/1563], Loss: 2.3056, Accuracy: 9.72%
Batch [100/1563], Loss: 2.3055, Accuracy: 9.56%
Batch [110/1563], Loss: 2.3059, Accuracy: 9.57%
Batch [120/1563], Loss: 2.3044, Accuracy: 9.79%
Batch [130/1563], Loss: 2.3051, Accuracy: 9.88%
Batch [140/1563], Loss: 2.3029, Accuracy: 10.11%
Batch [150/1563], Loss: 2.2990, Accuracy: 10.50%
Batch [160/1563], Loss: 2.2980, Accuracy: 10.57%
Batch [170/1563], Loss: 2.2963, Accuracy: 10.61%
Batch [180/1563], Loss: 2.2909, Accuracy: 11.23%
Batch [190/1563], Loss: 2.2830, Accuracy: 11.56%
Batch [200/1563], Loss: 2.2769, Accuracy: 11.94%
Batch [210/1563], Loss: 2.

In [ ]:
# 훈련 그래프
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Train Loss")
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Train Accuracy")
plt.title("Train Loss and Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Value")
plt.legend()
plt.savefig("train_results.png")  # 그래프 저장
plt.show()

# 모델 저장
torch.save(model.state_dict(), "inceptionv2_trained.pth")
print("Model saved to inceptionv2_trained.pth")